In [44]:
from evaluate_icsr_extraction import evaluate_icsr_from_dataset
from src import Icsr
from src.utils import get_matches
from datasets import load_dataset

In [20]:
path = './generated_eval_predictions.txt'
dataset_name = 'FAERS-PubMed/BioDEX-ICSR'
dataset_split = 'validation'

preds = []
with open(path, 'r') as fp:
    preds = fp.readlines()

metrics = evaluate_icsr_from_dataset(preds, dataset_name, dataset_split)

Using custom data configuration FAERS-PubMed--BioDEX-ICSR-40aa49fec6af4868
Found cached dataset parquet (/Users/kldooste/.cache/huggingface/datasets/FAERS-PubMed___parquet/FAERS-PubMed--BioDEX-ICSR-40aa49fec6af4868/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 264.71it/s]


In [7]:
metrics

((65.14026699423484, 59.396820453657234, 61.42847740089519), 26)

### Seriousness metric

In [21]:
preds_icsr = [Icsr.from_string(p) for p in preds]
preds_serious = [i.serious if i else '1' for i in preds_icsr]

dataset = load_dataset(dataset_name)
gold_strings = dataset[dataset_split]["target"]
gold_icsr = [Icsr.from_string(p) for p in gold_strings]
gold_serious = [i.serious if i else '1' for i in gold_icsr]

Using custom data configuration FAERS-PubMed--BioDEX-ICSR-40aa49fec6af4868
Found cached dataset parquet (/Users/kldooste/.cache/huggingface/datasets/FAERS-PubMed___parquet/FAERS-PubMed--BioDEX-ICSR-40aa49fec6af4868/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 315.19it/s]


In [27]:
from sklearn.metrics import classification_report

print(classification_report(gold_serious, preds_serious))

              precision    recall  f1-score   support

           1       0.95      0.98      0.96      2247
           2       0.43      0.23      0.30       160

    accuracy                           0.93      2407
   macro avg       0.69      0.60      0.63      2407
weighted avg       0.91      0.93      0.92      2407



In [28]:
print(classification_report(gold_serious, ['1'] * len(gold_serious)))

              precision    recall  f1-score   support

           1       0.93      1.00      0.97      2247
           2       0.00      0.00      0.00       160

    accuracy                           0.93      2407
   macro avg       0.47      0.50      0.48      2407
weighted avg       0.87      0.93      0.90      2407



/Users/kldooste/miniconda3/envs/biodex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kldooste/miniconda3/envs/biodex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kldooste/miniconda3/envs/biodex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [43]:
# The adverse event resulted in death, a life threatening condition, hospitalization, disability, congenital anomaly, or other serious condition"
def naive_baseline(fulltext):
    fulltext = fulltext.lower()
    if 'serious' in fulltext:
        return '1'
    elif 'death' in fulltext:
        return '1'
    elif 'hospitalization' in fulltext:
        return '1'
    elif 'threatening' in fulltext:
        return '1' 
    elif 'disability' in fulltext:
        return '1'
    elif 'anomaly' in fulltext:
        return '1'
    elif 'congenital' in fulltext:
        return '1'
    else:
        return '2'
    
naive_preds = [naive_baseline(f) for f in dataset[dataset_split]["fulltext_processed"]]
print(classification_report(gold_serious, naive_preds))

              precision    recall  f1-score   support

           1       0.94      0.64      0.76      2247
           2       0.08      0.42      0.13       160

    accuracy                           0.63      2407
   macro avg       0.51      0.53      0.45      2407
weighted avg       0.88      0.63      0.72      2407



### Do multiple reports per article share the seriousness value?

In [48]:
raw = load_dataset("FAERS-PubMed/raw_dataset")
matches = get_matches(raw['train'])

Using custom data configuration FAERS-PubMed--raw_dataset-0b83cc0b498dbbb2
Found cached dataset json (/Users/kldooste/.cache/huggingface/datasets/FAERS-PubMed___json/FAERS-PubMed--raw_dataset-0b83cc0b498dbbb2/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 53.48it/s]


In [54]:
uniform_seriousness = [len({r.serious for r in m.reports if r.serious != ''}) == 1 for m in matches]
print(100 * sum(uniform_seriousness) / len(uniform_seriousness))

95.91305142578601
